## Test the MIRI MRS distortion solution (standalone and JWST pipeline versions) ##

### First import the things that we need from the pipeline code ###

In [1]:
import os as os
import numpy as np
from numpy.testing import utils
from numpy.testing import assert_allclose
import pdb as pdb


Import the MIRI coordinates code from https://github.com/STScI-MIRI/miricoord and ensure that it is on the PYTHONPATH.  Also ensure that the local data directory environmental variable is set:

setenv MIRICOORD_DATA_DIR /YourLocalPathToData/ (this is where io will happen)

In [2]:
data_dir=os.path.expandvars('$MIRICOORD_DATA_DIR')

### First we'll look at the standalone distortion solution (mrs_tools) ###

In [3]:
import miricoord.miricoord.mrs.mrs_tools as mrst

Define a point on the MRS (0-indexed convention) and convert to v2,v3 for Ch1A

In [4]:
x,y=[50],[300]

Convert this pixel location to the MRS local alpha,beta,lam coordinates

In [5]:
values=mrst.xytoabl(x,y,'1A')

The returned values are a dictionary of different entries, including input pixels, output alpha/beta coordinates, wavelength (lam), and slice number/name.

In [6]:
values

{'x': array([50]),
 'y': array([300]),
 'alpha': array([-0.12611035]),
 'beta': array([1.77210144]),
 'lam': array([5.15139578]),
 'slicenum': array([21]),
 'slicename': array(['121A'], dtype='<U4')}

In [7]:
alpha,beta,lam=values['alpha'],values['beta'],values['lam']

In [8]:
alpha,beta,lam

(array([-0.12611035]), array([1.77210144]), array([5.15139578]))

Note that not every pixel on the detector actually maps to alpha,beta,lam since many pixels are between slices.  Therefore we have a way to 'trim' the inputs so that the returned values are only for such valid pixels.  Let's define an input array of pixels in which some map to slices and some do not.

In [9]:
x,y=[50,62,70,80,90],[500,510,520,10,950]

In [10]:
values=mrst.xytoabl(x,y,'1A',trim=1)

Note now that while we sent in five coordinates, only four got returned as one was trimmed.  The output values['x'] array is the trimmed version of the input x coordinate.

In [11]:
values

{'x': array([50, 70, 80, 90]),
 'y': array([500, 520,  10, 950]),
 'alpha': array([-0.23943881, -0.96215714,  1.49071685, -1.37745243]),
 'beta': array([ 1.77210144, -0.17721014, -0.17721014,  1.59489129]),
 'lam': array([5.32677646, 5.35591812, 4.8938786 , 5.69809225]),
 'slicenum': array([21, 10, 10, 20]),
 'slicename': array(['121A', '110A', '110A', '120A'], dtype='<U4')}

We can convert alpha,beta to v2,v3 as well:

In [12]:
v2,v3=mrst.abtov2v3(values['alpha'],values['beta'],'1A')

In [13]:
v2,v3

(array([-504.13917552, -504.57876517, -502.15932284, -505.2412633 ]),
 array([-320.46425433, -318.42639272, -318.78380022, -320.1144028 ]))

### Now let's test that the distortion transforms for all 12 channels/subbands match reference positions correctly: ###

In [14]:
mrst.testtransform()

Testing channel 1A
Testing channel 1B
Testing channel 1C
Testing channel 2A
Testing channel 2B
Testing channel 2C
Testing channel 3A
Testing channel 3B
Testing channel 3C
Testing channel 4A
Testing channel 4B
Testing channel 4C


### Now we'll look at the implementation of the distortion solution in the JWST pipeline (mrs_pipetools) ###

In [15]:
from astropy.modeling import models
from asdf import AsdfFile
from jwst import datamodels
from jwst.assign_wcs import miri

In [16]:
import miricoord.miricoord.mrs.mrs_pipetools as mrspt

Point to a simulated SHORT/A observation created with mirisim and reduced with the B7.1 pipeline through the end of assign_wcs

In [17]:
file=data_dir+'jwsttesting_may2018/20171218_160103_mirisim/det_images/det_image_seq1_MIRIFUSHORT_12SHORTexp1_assign_wcs.fits'

Read the datamodel from the file and print some information about it

In [18]:
im = datamodels.ImageModel(file)

In [19]:
print(im.meta.wcs)

   From       Transform    
---------- ----------------
  detector  detector_to_abl
alpha_beta     abl_to_v2v3l
      v2v3 CompoundModel760
     world             None


These are the available frames for the distortion transforms:

In [20]:
print(im.meta.wcs.available_frames)

['detector', 'alpha_beta', 'v2v3', 'world']


Define a point on the MRS and check that the v2,v3 is correct

In [21]:
alpha,beta,lam=0.0,0.0,5.0

In [22]:
v2,v3,ltemp=im.meta.wcs.transform("alpha_beta", "v2v3", alpha,beta,lam)

In [23]:
print(v2,v3)

-503.65447214471345 -318.742450697324


The V2,V3 location of the MRS 1A reference point should be -503.654, -318.742

Since this is an actual simulated exposure with a nominal telescope pointing, we can check the RA/DEC too.

The RA/DEC of the reference point is meant to be zero,zero in this simulation, check that it is

In [24]:
ra, dec, ltemp = im.meta.wcs.transform("v2v3", "world", v2,v3,lam)

In [25]:
print(ra,dec)

7.20594420019078e-16 -1.9224411181241964e-17


### Now let's test that the distortion transforms for all 12 channels/subbands match reference positions correctly:  ###

In [26]:
mrspt.testtransform()

Testing channel 1A
Testing channel 1B
Testing channel 1C
Testing channel 2A
Testing channel 2B
Testing channel 2C
Testing channel 3A
Testing channel 3B
Testing channel 3C
Testing channel 4A
Testing channel 4B
Testing channel 4C
